In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
!ls data

aac_intakes.csv          aac_intakes_outcomes.csv aac_outcomes.csv


In [8]:
df_out = pd.read_csv('data/aac_outcomes.csv')

In [9]:
df_in = pd.read_csv('data/aac_intakes.csv')

In [12]:
df_out.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80681 entries, 0 to 80680
Data columns (total 12 columns):
age_upon_outcome    80673 non-null object
animal_id           80681 non-null object
animal_type         80681 non-null object
breed               80681 non-null object
color               80681 non-null object
date_of_birth       80681 non-null object
datetime            80681 non-null object
monthyear           80681 non-null object
name                56116 non-null object
outcome_subtype     36893 non-null object
outcome_type        80667 non-null object
sex_upon_outcome    80679 non-null object
dtypes: object(12)
memory usage: 7.4+ MB


In [13]:
df_in.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80187 entries, 0 to 80186
Data columns (total 12 columns):
age_upon_intake     80187 non-null object
animal_id           80187 non-null object
animal_type         80187 non-null object
breed               80187 non-null object
color               80187 non-null object
datetime            80187 non-null object
datetime2           80187 non-null object
found_location      80187 non-null object
intake_condition    80187 non-null object
intake_type         80187 non-null object
name                55603 non-null object
sex_upon_intake     80186 non-null object
dtypes: object(12)
memory usage: 7.3+ MB


In [20]:
df = df_in.merge(df_out, on='animal_id', suffixes=('_in', '_out'))

In [22]:
df.columns.values

array(['age_upon_intake', 'animal_id', 'animal_type_in', 'breed_in',
       'color_in', 'datetime_in', 'datetime2', 'found_location',
       'intake_condition', 'intake_type', 'name_in', 'sex_upon_intake',
       'age_upon_outcome', 'animal_type_out', 'breed_out', 'color_out',
       'date_of_birth', 'datetime_out', 'monthyear', 'name_out',
       'outcome_subtype', 'outcome_type', 'sex_upon_outcome'],
      dtype=object)

## Drop columns that will stay the same across out and in

In [30]:
df = df.drop(labels=['animal_type_in', 'breed_in', 'color_in', 'datetime2', 'name_in'], axis=1)

In [34]:
df.columns.values

array(['age_upon_intake', 'animal_id', 'datetime_in', 'found_location',
       'intake_condition', 'intake_type', 'sex_upon_intake',
       'age_upon_outcome', 'animal_type_out', 'breed_out', 'color_out',
       'date_of_birth', 'datetime_out', 'monthyear', 'name_out',
       'outcome_subtype', 'outcome_type', 'sex_upon_outcome'],
      dtype=object)

In [35]:
df.head()

,age_upon_intake,animal_id,datetime_in,found_location,intake_condition,intake_type,sex_upon_intake,age_upon_outcome,animal_type_out,breed_out,color_out,date_of_birth,datetime_out,monthyear,name_out,outcome_subtype,outcome_type,sex_upon_outcome
0,8 years,A706918,2015-07-05T12:59:00.000,9409 Bluegrass Dr in Austin (TX),Normal,Stray,Spayed Female,8 years,Dog,English Springer Spaniel,White/Liver,2007-07-05T00:00:00,2015-07-05T15:13:00,2015-07-05T15:13:00,Belle,NaN,Return to Owner,Spayed Female
1,11 months,A724273,2016-04-14T18:43:00.000,2818 Palomino Trail in Austin (TX),Normal,Stray,Intact Male,1 year,Dog,Basenji Mix,Sable/White,2015-04-17T00:00:00,2016-04-21T17:17:00,2016-04-21T17:17:00,Runster,NaN,Return to Owner,Neutered Male
2,4 weeks,A665644,2013-10-21T07:59:00.000,Austin (TX),Sick,Stray,Intact Female,4 weeks,Cat,Domestic Shorthair Mix,Calico,2013-09-21T00:00:00,2013-10-21T11:39:00,2013-10-21T11:39:00,NaN,Partner,Transfer,Intact Female
3,4 years,A682524,2014-06-29T10:38:00.000,800 Grove Blvd in Austin (TX),Normal,Stray,Neutered Male,4 years,Dog,Doberman Pinsch/Australian Cattle Dog,Tan/Gray,2010-06-29T00:00:00,2014-07-02T14:16:00,2014-07-02T14:16:00,Rio,NaN,Return to Owner,Neutered Male
4,2 years,A743852,2017-02-18T12:46:00.000,Austin (TX),Normal,Owner Surrender,Neutered Male,2 years,Dog,Labrador Retriever Mix,Chocolate,2015-02-18T00:00:00,2017-02-21T17:44:00,2017-02-21T17:44:00,Odin,NaN,Return to Owner,Neutered Male


In [37]:
df.rename(mapper={'animal_type_out': 'animal_type', 'breed_out': 'breed', 'color_out': 'color',
                  'name_out': 'name'}, inplace=True)

In [39]:
df.drop(labels=['monthyear'], axis=1, inplace=True)

In [40]:
df.columns.values

array(['age_upon_intake', 'animal_id', 'datetime_in', 'found_location',
       'intake_condition', 'intake_type', 'sex_upon_intake',
       'age_upon_outcome', 'animal_type_out', 'breed_out', 'color_out',
       'date_of_birth', 'datetime_out', 'name_out', 'outcome_subtype',
       'outcome_type', 'sex_upon_outcome'], dtype=object)

In [41]:
df.shape

(100230, 17)